# Optimizers

In this part we are going to deal with how to do stochastic gradients. There are different type of gradient methods to address the problems arising during optimization. We will cover the different optimizers in the order of their appearance. During that we will uncover the problems they solve and approach with which they solve it. 

Generally, we have a loss function and we want to find its minimum point. Each of the loss functions can be derived as a sum of sample loss functions:

\begin{equation}
L_\Theta\left(y_t, y_p\right) = \frac{1}{n}\sum^n_i { l_\Theta\left(y^{(i)}_t, y^{(i)}_p \right) }.
\end{equation}

The goal is to minimize the $L_\Theta$ function. This requires to calculate the $l_\Theta$ for all the samples which can take a lot of time. To avoid that, the common approach is to draw a batch of samples from the training data and calculate the loss (and what is more important the gradient) according to that. The larger the batch, the closer it is to the gradient calculated on the whole dataset. When the batch size becomes smaller the speed of calculating the gradient becomes faster but also noisier. Optimizers are different in the way how they update the parameters according to the gradient. To make it simple what one can do with the optimization process is to accumulate the gradients or change the learning rate adaptively. The research community have found clever ways to do that and now we have different optimizer strategies.

## SGD 

SGD means Stochastic Gradient Descent. Originally, SGD means that the batch size is one but in practice it has such a high variance that it is not a widespreaded approach, instead we use the Minibatch Gradient Descent, which has the principles we discussed in the previous paragraph. Nowadays, the term SGD means the Minibatch Gradient Descent. But furthermore we think of vanilla-SGD under the term SGD. Vanilla in this context means that the gradients are directly applied on the parameters, there is nothing to do with the learning rate and the gradients.

The formula:

\begin{equation}
\Theta \leftarrow \Theta - \alpha \cdot \frac{\partial L}{\partial \Theta} 
\end{equation}

In [1]:
# an example for the convergence behavior

## Momentum

## Adam

[Adam](https://arxiv.org/pdf/1412.6980.pdf)

## Adadelta

[Adadelta](https://arxiv.org/pdf/1212.5701.pdf)

## RMSProp

## Adagrad

### Further references

[Overview of optimizers by Ruder](http://ruder.io/optimizing-gradient-descent/index.html#rmsprop) <br/>